<a href="https://colab.research.google.com/github/natalievolk/LearnAI/blob/main/LearnAI_Take_Four.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [80]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import Adam, SGD
from torch.utils.data import Dataset, DataLoader,TensorDataset
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from PIL import Image
import cv2
import requests
from io import BytesIO
from tqdm.notebook import tqdm

In [81]:
train_txt = "https://raw.githubusercontent.com/natalievolk/LearnAI/training/train.txt"
test_txt = "https://raw.githubusercontent.com/natalievolk/LearnAI/training/test.txt"

In [82]:
class ImagesDataset(Dataset):
    def __init__(self, text_file, transform):
        self.df = pd.read_csv(text_file, sep=' ')
        self.df.columns = ['jpg_url', 'classification']
        self.transform = transform

    def __len__(self):
        return 1600 #len(self.df)

    def __getitem__(self, idx):
        response = requests.get(self.df['jpg_url'][idx])
        img = Image.open(BytesIO(response.content)).resize((400, 400))
        pix = np.array(img)
        try:
          pix = self.transform(pix)
        except:
          pix = np.stack((np.array(pix, copy=True), np.array(pix, copy=True), np.array(pix, copy=True)), axis=2)
          pix = self.transform(pix)
 
        return pix, int(self.df['classification'][idx])

In [83]:
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

train_dataset = ImagesDataset(train_txt,transform)
train_dataloader = DataLoader(train_dataset, batch_size=64, num_workers=2)

test_dataset = ImagesDataset(test_txt,transform)
test_dataloader = DataLoader(test_dataset, batch_size=64, num_workers=2)

In [84]:
class Net(nn.Module):   
    def __init__(self):
        super(Net, self).__init__()
        self.pool = nn.MaxPool2d(2,2)
        self.conv1 = nn.Conv2d(3,16,5,2)
        self.conv2 = nn.Conv2d(16,128,7,2)
        self.conv3 = nn.Conv2d(128,512,9,2)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(512*4*4, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 30)
        self.softmax = nn.Softmax()

    # Defining the forward pass    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        # print(x.shape)
        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        # x = self.softmax(x)
        return x

net = Net()
optimizer = Adam(net.parameters(), lr=0.01)
criterion = CrossEntropyLoss()
if torch.cuda.is_available():
    net = net.cuda()
    criterion = criterion.cuda()
device = torch.device('cuda:0')

In [ ]:
#Training Stage
for epoch in range(5):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(tqdm(train_dataloader)):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        # predictions = outputs.argmax(dim=1)
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        print(i,loss.item())

    print('[%d] loss: %f' %
              (epoch + 1, running_loss))

print('Finished Training')
#minibatch, cnn parameter, epoch, learning rate,   

0 3.438152313232422
1 370.7009582519531
2 4.498750686645508
3 3.4084527492523193
4 3.394514799118042
5 3.3952605724334717
6 3.397800922393799
7 4.031215190887451
8 3.382948875427246
9 3.3395957946777344
10 3.3336284160614014
11 11.047839164733887
12 3.34310245513916
13 3.297255516052246
14 3.290304660797119
15 3.3676795959472656
16 3.3468878269195557
17 3.32498836517334
18 3.245143413543701
19 3.3097832202911377
20 3.3024253845214844
21 3.313300371170044
22 3.2914462089538574
23 3.367724895477295
24 3.315225839614868

[1] loss: 460.484385


0 3.1760897636413574
1 3.3733277320861816
2 3.282656192779541
3 3.2613048553466797
4 3.3294034004211426
5 3.2957067489624023
6 3.292917490005493
7 3.2962586879730225
8 3.133632183074951
9 3.1877834796905518
10 3.341212272644043
11 3.2929773330688477
12 3.3663077354431152
13 3.211031198501587
14 3.240989923477173
15 3.3517398834228516
16 3.2732326984405518
17 3.335547924041748
18 3.2023067474365234
19 3.272672176361084
20 3.24334979057312
21 3.2772107124328613
22 3.2336394786834717
23 3.337306499481201
24 3.2968974113464355

[2] loss: 81.905502


0 3.12355637550354
1 3.390749216079712
2 3.2290282249450684
3 3.2252869606018066
4 3.3023548126220703
5 3.2863962650299072
6 3.2908074855804443
7 3.2688095569610596
8 3.127095937728882
9 3.1611430644989014
10 3.297834634780884
11 3.2839715480804443
12 3.326770544052124
13 3.1845455169677734
14 3.248241662979126
15 3.3544654846191406
16 3.2609527111053467
17 3.348555564880371


In [ ]:
dataiter = iter(train_dataloader)
images, labels = dataiter.next()

# print images
imshow(torchvision.utils.make_grid(images))
print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(4)))